In [79]:
!pwd

/global/cscratch1/sd/zonca/pico-simulations/full_focal_plane


In [80]:
import pandas as pd
import numpy as np

In [81]:
original_pixels = pd.read_csv("original_pixels.txt", delim_whitespace=True, header=None)

In [82]:
pixels = pd.read_csv("pixels.txt", delim_whitespace=True, header=None)

In [83]:
bands = pd.read_csv("original_bands.txt", delim_whitespace=True).set_index("band")

## Scale the white noise level down

The sparse focal plane means we are reducing the number of pixels per wafer to 7 for all bands.
Therefore the number of detectors is reduced by a factor of $\dfrac{7}{original\_pixel\_per\_wafer}$, for example for band 9 it is a factor of $\dfrac{7}{19}$.

We would still like to get output maps with the same noise properties of the original focal plane, therefore we reduce the `NET` by multiplying by $\sqrt{\dfrac{7}{original\_pixel\_per\_wafer}}$

In [84]:
import pandas

def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    
    From https://gist.github.com/jlln/338b4b0b55bd6984f883
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pandas.DataFrame(new_rows)
    return new_df

In [85]:
original_pixels = splitDataFrameList(original_pixels, 1, ",").set_index(1)

In [86]:
pixels = splitDataFrameList(pixels, 1, ",").set_index(1)

In [87]:
NET_scale_factor = np.sqrt(pixels[2] / original_pixels[2])

In [88]:
NET_scale_factor

1
1     1.000000
3     1.000000
5     1.000000
2     1.000000
4     1.000000
6     1.000000
7     0.606977
9     0.606977
11    0.606977
8     0.434959
10    0.434959
12    0.434959
13    0.203519
15    0.203519
17    0.203519
14    0.234772
16    0.234772
18    0.234772
19    0.234772
20    0.234772
21    0.234772
Name: 2, dtype: float64

In [89]:
(pd.to_numeric(bands["NET"][1:]) * NET_scale_factor)

1       94.700000
10      13.962179
11      16.206285
12      11.439417
13      10.155615
14      13.147258
15       8.486756
16      12.982917
17      14.042834
18      33.102918
19     108.112720
2       86.600000
20     428.694521
21    2536.974732
3       54.900000
4       50.200000
5       40.300000
6       37.100000
7       34.294199
8       22.922331
9       20.515822
dtype: float64

In [90]:
bands["NET"] = pd.to_numeric(bands["NET"][1:]) * NET_scale_factor

In [91]:
bands

,nu_center,nu_width,fwhm,pol_weight,NET,f_knee,f_min,alpha
band,,,,,,,,
none,GHz,GHz,arcmin,uK_arcmin,NaN,mHz,mHz,none
1,21.0,5.2,38.4,16.3,94.700000,10.0,0.01,1.0
2,25.0,6.3,32.0,11.7,86.600000,10.0,0.01,1.0
3,30.0,7.5,28.3,7.8,54.900000,10.0,0.01,1.0
4,36.0,9.0,23.6,5.6,50.200000,10.0,0.01,1.0
5,43.2,10.8,22.2,5.4,40.300000,10.0,0.01,1.0
6,51.8,13.0,18.4,4.0,37.100000,10.0,0.01,1.0
7,62.2,15.6,12.8,3.9,34.294199,10.0,0.01,1.0
8,74.6,18.7,10.7,3.2,22.922331,10.0,0.01,1.0


In [92]:
bands.to_csv("bands.txt", sep=" ")

## Prepare the slurm scripts

We need to use a template and fill in what we need to run the other 20 bands.

* change the band and the output folder for each band
* set the sampling frequency to 63 for all channels beside the lowest bands which already have a lower rate
* scale the number of nodes based on the total number of wafers



In [100]:
wafer_count = pd.read_csv("wafer_count.txt", delim_whitespace=True, header=None).set_index(0).to_dict()[1]

In [101]:
wafer_count

{'A': 10, 'B': 14, 'C': 18, 'D': 12, 'E': 3, 'F': 3, 'GHI': 1}

I already ran band 9 which is of type `C` which has the highest number of channels total.
and this used 730 nodes.

We want to scale the total number of nodes by the ratio of the wafer count.

In [132]:
from jinja2 import Template

In [142]:
with open("pico_slurm.sh.j2") as f:
    slurm_template = Template(f.read())

In [143]:
for band in range(1,22):
    band_str = str(band)
    pix_type = pixels.loc[str(band)][0]
    samplerate = pixels.loc[str(band)][3]
    pix_wafer_count = wafer_count.get(pix_type, 1)
    pix_wafer_scale = pix_wafer_count / 18
    nodes = int(np.ceil(730 * pix_wafer_scale /2)*2)
    # need even nodes
    rendered = slurm_template.render(nodes=nodes, samplerate=samplerate, band=band_str)
    with open(f"slurm_scripts/{band:02d}.slurm", "w") as out_f:
        out_f.write(rendered)
    print(f"Band {band}, {pix_type}, {pix_wafer_count} wafers has scale of {pix_wafer_scale}, {samplerate} Hz NODES {nodes}")

Band 1, A, 10 wafers has scale of 0.5555555555555556, 42.0 Hz NODES 405
Band 2, B, 14 wafers has scale of 0.7777777777777778, 51.0 Hz NODES 567
Band 3, A, 10 wafers has scale of 0.5555555555555556, 42.0 Hz NODES 405
Band 4, B, 14 wafers has scale of 0.7777777777777778, 51.0 Hz NODES 567
Band 5, A, 10 wafers has scale of 0.5555555555555556, 42.0 Hz NODES 405
Band 6, B, 14 wafers has scale of 0.7777777777777778, 51.0 Hz NODES 567
Band 7, C, 18 wafers has scale of 1.0, 63.0 Hz NODES 730
Band 8, D, 12 wafers has scale of 0.6666666666666666, 63.0 Hz NODES 486
Band 9, C, 18 wafers has scale of 1.0, 63.0 Hz NODES 730
Band 10, D, 12 wafers has scale of 0.6666666666666666, 63.0 Hz NODES 486
Band 11, C, 18 wafers has scale of 1.0, 63.0 Hz NODES 730
Band 12, D, 12 wafers has scale of 0.6666666666666666, 63.0 Hz NODES 486
Band 13, E, 3 wafers has scale of 0.16666666666666666, 63.0 Hz NODES 121
Band 14, F, 3 wafers has scale of 0.16666666666666666, 63.0 Hz NODES 121
Band 15, E, 3 wafers has scale o

In [144]:
rm slurm_scripts/10.slurm

In [145]:
ls slurm_scripts/

01.slurm  04.slurm  07.slurm  11.slurm  14.slurm  17.slurm  20.slurm
02.slurm  05.slurm  08.slurm  12.slurm  15.slurm  18.slurm  21.slurm
03.slurm  06.slurm  09.slurm  13.slurm  16.slurm  19.slurm
